<a href="https://colab.research.google.com/github/HenryLau08/adsai-mlops-2025/blob/main/MLOPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install kaggle

In [ ]:
# %pip install ultralytics

In [ ]:
# %pip install mlflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:

# imports
import os
import cv2
from roboflow import Roboflow
import shutil
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image


robo_api_key = "sGinQr2B54FkDko52rnT" # Set your roboflow api key
data_dir = "/content/drive/MyDrive/Colab Notebooks/data"
os.makedirs(data_dir, exist_ok=True)
os.makedirs("/root/.config/kaggle/", exist_ok=True)

# shutil.move("kaggle.json", "/root/.config/kaggle/") # manually upload your kaggle.json file and run it once
# !chmod 600 /root/.config/kaggle/kaggle.json
import kaggle


Groepsnaam

Groepsleden

michal
quincy
henry
moh
jayden

Opdrachtgever

Tesla


# 1. INLEIDING

opdrachtbeschrijving


Beschrijf het doel van de datapijplijn en geef een korte samenvatting van de stappen die je gaat volgen.


# 2. Data Ingestion
Beschrijf hoe de data wordt ingeladen vanuit verschillende bronnen (Kaggle, Google Images, live camera feeds).


Kaggle Datasets

Beschrijf hoe je de datasets van Kaggle downloadt en inlaadt.

Gebruik bijvoorbeeld de Kaggle API om de datasets te downloaden.

Google Images

Beschrijf hoe je afbeeldingen van Google Images haalt via web scraping of een API.

Leg uit hoe je de afbeeldingen opslaat en organiseert.

Live Camera Feeds

Beschrijf hoe je live videofeeds verwerkt voor realtime voertuigdetectie.

Leg uit hoe je frames uit de videofeeds haalt en deze gebruikt als input voor het model.



## 2.1 Kaggle Datasets

In [ ]:
kaggle_path = str(data_dir + '/kaggle')
os.makedirs(kaggle_path, exist_ok=True)

kaggle.api.dataset_download_files("sakshamjn/vehicle-detection-8-classes-object-detection", path=kaggle_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection


In [8]:
images_path = "/content/drive/MyDrive/Colab Notebooks/data/kaggle/train/images"
labels_path = "/content/drive/MyDrive/Colab Notebooks/data/kaggle/train/labels"

images = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

In [ ]:
def extract_labels_and_bboxes(label_file):
    labels_and_bboxes = []
    with open(label_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Split by space to get label and bbox
            parts = line.strip().split(' ')
            label = int(parts[0])  # label is the first part
            bbox = [float(x) for x in parts[1:]]  # bbox is the rest (x_center, y_center, width, height)
            labels_and_bboxes.append((label, bbox))
    return labels_and_bboxes

# Create an empty list to store all the extracted data
data = []

# Loop through each image and corresponding label file
for img, lbl in zip(images, labels):
    img_path = os.path.join(images_path, img)
    lbl_path = os.path.join(labels_path, lbl)

    # Extract the labels and bounding boxes from the label file
    labels_and_bboxes = extract_labels_and_bboxes(lbl_path)

    for label, bbox in labels_and_bboxes:
        # Add the data for each label and bounding box into the list
        data.append({
            'image_path': img_path,
            'label_path': lbl_path,
            'label': label,
            'x_center': bbox[0],
            'y_center': bbox[1],
            'width': bbox[2],
            'height': bbox[3]
        })

# Create DataFrame from the list of dictionaries
df_kaggle = pd.DataFrame(data)
csv_path = os.path.join(data_dir, 'kaggle_data.csv')
df_kaggle.to_csv(csv_path, index=False)

In [9]:
csv_path = os.path.join(data_dir, 'kaggle_data.csv')
df_kaggle = pd.read_csv(csv_path)

In [10]:
display(df_kaggle.head(10))

,image_path,label_path,label,x_center,y_center,width,height
0,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,2,0.521635,0.295673,0.043269,0.086538
1,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,2,0.570913,0.604567,0.098558,0.242788
2,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,7,0.395433,0.537260,0.156250,0.382212
3,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,4,0.930288,0.747596,0.052885,0.158654
4,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,4,0.618990,0.290865,0.021635,0.057692
5,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,7,0.459135,0.212740,0.067308,0.185096
6,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,4,0.102163,0.419471,0.103365,0.122596
7,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,2,0.497596,0.675481,0.096154,0.259615
8,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,7,0.490385,0.403846,0.052885,0.168269
9,/content/drive/MyDrive/Colab Notebooks/data/ka...,/content/drive/MyDrive/Colab Notebooks/data/ka...,7,0.721154,0.784856,0.250000,0.396635


## 2.2 Google Images


## 2.3 Live Camera Feeds


# 3. Data Opschonen en Transformeren

Beschrijf de stappen voor data cleaning, data augmentation en data transformation.

In deze sectie beschrijf je de stappen voor het opschonen en transformeren van de data. Gebruik markdown-cellen om elke stap te beschrijven en code-cellen om de transformaties uit te voeren.

Data Cleaning

Beschrijf hoe je de afbeeldingen opschoont (bijvoorbeeld door ze allemaal naar hetzelfde formaat te converteren).

Voer kleurnormalisatie uit en pas cropping en padding toe waar nodig.

Data Augmentation

Beschrijf hoe je data augmentation toepast om de dataset te vergroten en het model robuuster te maken.

Gebruik technieken zoals rotatie, spiegeling, schalen, translatie, shearing en cropping.

Data Transformation

Beschrijf hoe je de afbeeldingen transformeert om ze geschikt te maken voor machine learning (bijvoorbeeld normalisatie en kleurtransformaties).

## 3.1 Data Cleaning

In [13]:
image_sizes = set()

for path in df_kaggle['image_path']:
    try:
        with Image.open(path) as img:
            size = img.size  # (width, height)
            image_sizes.add(size)
    except Exception as e:
        print(f"Fout bij openen van {path}: {e}")

# Toon alle unieke afmetingen
print("Unieke afbeeldingsgroottes (width x height):")
for size in sorted(image_sizes):
    print(size)

Unieke afbeeldingsgroottes (width x height):
(382, 382)
(384, 384)
(388, 388)
(390, 390)
(392, 392)
(394, 394)
(396, 396)
(398, 398)
(400, 400)
(402, 402)
(404, 404)
(406, 406)
(408, 408)
(410, 410)
(412, 412)
(416, 416)
(640, 480)
(640, 640)


## 3.2 Data Augmentation

## 3.3 Data Transformation

# 4. Machine Learning Pipeline

Beschrijf hoe het objectdetectiemodel wordt getraind en geëvalueerd.


In deze sectie beschrijf je hoe je het objectdetectiemodel traint en evalueert. Gebruik markdown-cellen om elke stap te beschrijven en code-cellen om het model te trainen en te evalueren.

Objectdetectie Model

Beschrijf hoe je een objectdetectiemodel (bijvoorbeeld YOLO, Faster R-CNN of EfficientDet) traint op de dataset.

Leg uit hoe je het model evalueert met behulp van metrieken zoals precisie, recall en F1-score.

Image Captioning

Beschrijf eventueel hoe je een image captioning-model toevoegt om beschrijvingen van gedetecteerde voertuigen te genereren.



## 4.1 Train-Test Split

## 4.2 Model Bouwen

## 4.3 Model Trainen

## 4.4 Model Evalueren

# 5. Deployment

Beschrijf hoe het model wordt gedeployed op edge-apparaten en in de cloud.

In deze sectie beschrijf je hoe je het model deployt op edge-apparaten en in de cloud. Gebruik markdown-cellen om de deploymentstrategie te beschrijven.

Edge Deployment

Beschrijf hoe je het model implementeert op edge-apparaten zoals slimme camera's of embedded systemen.

Leg uit hoe je ervoor zorgt dat het model realtime voertuigdetectie kan uitvoeren.

Cloud Deployment

Beschrijf hoe je het geavanceerde deep learning-model in de cloud deployt voor voertuigclassificatie.



## 5.1 Edge Deployment

## 5.2 Cloud Deployment

# 6. Monitoring en Feedback

Beschrijf hoe de prestaties van het model worden gemonitord en hoe het model wordt hertraind op basis van nieuwe data.

In deze sectie beschrijf je hoe je de prestaties van het model monitort en hoe je het model hertraind op basis van nieuwe data. Gebruik markdown-cellen om de monitoringstrategie te beschrijven.

Data Drift Detection

Beschrijf hoe je veranderingen in de inkomende data detecteert en hoe je hierop reageert.

Model Performance Monitoring

Beschrijf hoe je de nauwkeurigheid, recall en precision van het model continu monitort.

Leg uit hoe je automatisch logging van verkeerde voorspellingen implementeert.

Hertraining

Beschrijf hoe je het model periodiek hertraind op basis van nieuwe data en feedback.

7. Conclusie


## 6.1 Data Drift Detection

## 6.2 Hertraining

# 7. Conclusie

Geef een samenvatting van de datapijplijn en bespreek eventuele uitdagingen of verbeterpunten.

In deze sectie geef je een samenvatting van de datapijplijn en bespreek je eventuele uitdagingen of verbeterpunten. Gebruik markdown-cellen om de conclusie te schrijven.




# 8. Bronvermelding